In [320]:
import pandas as pd
from openbb import obb
from datetime import date,datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
import calendar
import requests

obb.user.preferences.output_type = 'dataframe'

In [326]:
url = 'https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v2/accounting/od/avg_interest_rates?sort=-record_date&format=json&page[number]=1&page[size]=1'

response = requests.get(url)
data = response.json()
avg_interest_treasury_rate = round(float(data['data'][0]['avg_interest_rate_amt']),1)

In [327]:
tickers = ['AAPL', 'NVDA', 'googl']

todays_date = date.today()

yesterday = todays_date - timedelta(1)

five_years = todays_date - relativedelta(years = 5)

In [330]:
stats = obb.equity.ownership.share_statistics(symbol='AAPL', provider='yfinance', start_date = five_years )
fundamental = obb.equity.fundamental.income(symbol='AAPL', provider='yfinance', start_date = five_years)


In [331]:
days = 365
years = list(range(2020, 2026))
months= list(range(1,13))

def days_in_month(years, months):
    days_dict = {}
    for year in years:
        for month in months:
            days = calendar.monthrange(year, month)[1]
            days_dict[(year, month)] = days
    return days_dict

days_per_month = days_in_month(years, months)
number_of_days = sum(days_per_month.values())
number_of_days

2192

## Simple Returns

$$
\frac{p_1}{p_{t-1}} - 1 
$$

## Compound Return

$$
\log \frac{p_t}{p_{t-1}}
$$

## CAGR (Compound Annual Growth Rate) ￼

$$ 
(\frac{endingvalue}{beginning value})^\frac{1}{n} - 1
$$
###### You want the beginning value to understand the initial starting point of the investment, along with the ending value as the most recent point of the investment

###### It's always good to be able to see how much an investment grows given annual rate, ignoring volatility rate

### Sharpe Ratio

$$
\frac{R_p - R_f}{\sigma_p}
$$


#### Alpha

$$ 
\alpha = R_i - [R_f + \beta_i (R_M + R_f)]
$$


#### Delta

$$
\Delta = \frac{\partial V}{\partial S}
$$

##### More in regards to options, looks at the change in dollar

In [343]:
def closing_data(tickers):
    tickers = [ticker.upper() for ticker in tickers]
    stocks_collections = {}
    def market_performance(stock = 'SPY'):
        data = []
        dataframe = pd.DataFrame()
        # more columns
        dataframe = obb.equity.price.historical(stock,provider = 'yfinance', start_date = five_years)
        dataframe['market_returns'] = (dataframe.loc[:,'close'] / dataframe.loc[:,'close'].shift()) - 1
        dataframe['compound_returns'] = np.log((dataframe.loc[:, 'close'] / dataframe.loc[:,'close'].shift()))  
        dataframe['CAGR_Percent'] = (dataframe.iloc[-1]['close'] / dataframe.iloc[0]['close']) ** (1/5) - 1
        dataframe['annualized_volatility'] = ((dataframe.loc[:,'close'] / dataframe.loc[:,'close'].shift())-1).std() * np.sqrt(252)
        return dataframe.dropna()
    spy = market_performance()
    window = 252
    data = []
    dataframe = pd.DataFrame()
    for ticker in tickers:
        # more columns
        dataframe = obb.equity.price.historical(ticker,provider = 'yfinance', start_date = five_years)
        outsource = obb.equity.ownership.share_statistics(symbol=ticker, provider='yfinance', start_date = five_years)
        fundamental = obb.equity.fundamental.income(symbol= ticker, provider='yfinance', start_date = five_years)
        
        # dataframe['market_cap'] = dataframe['close'] * outsource['outstanding_shares']
        dataframe['simple_returns'] = (dataframe.loc[:,'close'] / dataframe.loc[:,'close'].shift()) - 1
        dataframe['compound_returns'] = np.log((dataframe.loc[:, 'close'] / dataframe.loc[:,'close'].shift()))  
        dataframe['CAGR_Percent'] = (dataframe.iloc[-1]['close'] / dataframe.iloc[0]['close']) ** (1/5) - 1
        dataframe['annualized_volatility'] = ((dataframe.loc[:,'close'] / dataframe.loc[:,'close'].shift())-1).std() * np.sqrt(252)
        def calc_beta(dataframe):
            return dataframe['simple_returns'].cov(spy['market_returns']) / spy['market_returns'].var()
        def joining_two_rates(dataframe, market = spy):
            total = pd.concat([dataframe, spy], axis = 1)
            return total
        total_data = joining_two_rates(dataframe, market = spy)[['simple_returns', 'market_returns']]

        returns = total_data['simple_returns'].rolling(window).cov(total_data['market_returns']).dropna()
        dataframe['beta'] =  returns / total_data['market_returns'].rolling(window).var()
        dataframe['alpha'] = dataframe['simple_returns'] - (avg_interest_treasury_rate + dataframe['beta'] * (total_data['market_returns'] - avg_interest_treasury_rate)) 
        
        stocks_collections[ticker] = dataframe.dropna()
        
        
    return stocks_collections
    
data = closing_data(tickers)
apple = data['AAPL']   

stocks = data.keys()

apple

,open,high,low,close,volume,split_ratio,dividend,simple_returns,compound_returns,CAGR_Percent,annualized_volatility,beta,alpha
date,,,,,,,,,,,,,
2021-08-10,146.440002,147.710007,145.300003,145.600006,69023100,0.0,0.0,-0.003354,-0.003360,0.143116,0.294132,1.512597,2.198933
2021-08-11,146.050003,146.720001,145.529999,145.860001,48493500,0.0,0.0,0.001786,0.001784,0.143116,0.294132,1.505395,2.171244
2021-08-12,146.190002,149.050003,145.839996,148.889999,72282600,0.0,0.0,0.020773,0.020561,0.143116,0.294132,1.499436,2.163853
2021-08-13,148.970001,149.440002,148.270004,149.100006,59375000,0.0,0.0,0.001410,0.001409,0.143116,0.294132,1.502211,2.158182
2021-08-16,148.539993,151.190002,146.470001,151.119995,103296000,0.0,0.0,0.013548,0.013457,0.143116,0.294132,1.502755,2.171856
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-01,210.869995,213.580002,201.500000,202.380005,104434500,0.0,0.0,-0.025004,-0.025322,0.143116,0.294132,1.196814,0.840913
2025-08-04,204.509995,207.880005,201.679993,203.350006,75109300,0.0,0.0,0.004793,0.004782,0.143116,0.294132,1.193722,0.819652
2025-08-05,203.399994,205.339996,202.160004,202.919998,44155100,0.0,0.0,-0.002115,-0.002117,0.143116,0.294132,1.193424,0.835658


In [ ]:
def downloading_option_data(tickers):
    tickers = [ticker.upper() for ticker in tickers]

    hasher = {}
    
    for ticker in tickers:
        dataframe = obb.derivatives.options.chains(ticker, provider="yfinance", start_date = todays_date).dropna()
        hasher[ticker] = dataframe.set_index['expiration', 'strike', 'option_type', 'symbol']
    return hasher
    
    

data = downloading_option_data(tickers)

In [ ]:
data['AAPL']